In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_daily
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH ranked_data AS (
  SELECT
    knumh cond_rec_num,
    kappl app,
    recordmode record_mode,
    kopos seq_of_cond,
    kschl cond_type,
    knumt num_text,
    stfkz scale_type,
    kzbzg scale_indicator,
    kstbm cond_scale_qty,
    konms cond_scale_uom,
    kstbw scale_value,
    konws scale_currency,
    krech cal_type_cond,
    kbetr rate,
    konwa rate_unit,
    kpein cond_pricing_unit,
    kmein cond_unit,
    prsch price_level,
    kumza numerator,
    kumne denominator,
    meins base_uom,
    mxwrt lower_limit,
    gkwrt upper_limit,
    pkwrt planned_cond_value,
    fkwrt,
    rswrt,
    kwaeh cond_currency,
    ukbas planned_cond_basic,
    kznep cond_excl_indicator,
    kunnr customer_number,
    lifnr,
    mwsk1 tax_code,
    loevm_ko del_indicator,
    zaehk_ind cond_item_id,
    bomat,
    kbrue accr_amount,
    kspae,
    bosta,
    knuma_pi,
    knuma_ag sale_deal,
    knuma_sq sale_quote,
    valtg add_days,
    valdt fix_date,
    zterm,
    anzauf,
    mikbas,
    mxkbas,
    komxwrt max_value,
    klf_stg,
    klf_kal,
    vkkal,
    aktnr promotion,
    knuma_bo agreement,
    mwsk2 withholding_tax_code,
    vertt,
    vertn,
    vbewa,
    mdflg,
    kfrst,
    uasta status_bussiness,
    ecrtt,
    ecrtn,
    ecewa flow_type,
    waers currency_key,
    ROW_NUMBER() OVER (PARTITION BY knumh, kappl ORDER BY proc_date) AS row_num
  FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb31
  WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb31)
)
SELECT
  cond_rec_num,
  app,
  record_mode,
  seq_of_cond,
  cond_type,
  num_text,
  scale_type,
  scale_indicator,
  cond_scale_qty,
  cond_scale_uom,
  scale_value,
  scale_currency,
  cal_type_cond,
  rate,
  rate_unit,
  cond_pricing_unit,
  cond_unit,
  price_level,
  numerator,
  denominator,
  base_uom,
  lower_limit,
  upper_limit,
  planned_cond_value,
  fkwrt,
  rswrt,
  cond_currency,
  planned_cond_basic,
  cond_excl_indicator,
  customer_number,
  lifnr,
  tax_code,
  del_indicator,
  cond_item_id,
  bomat,
  accr_amount,
  kspae,
  bosta,
  knuma_pi,
  sale_deal,
  sale_quote,
  add_days,
  fix_date,
  zterm,
  anzauf,
  mikbas,
  mxkbas,
  max_value,
  klf_stg,
  klf_kal,
  vkkal,
  promotion,
  agreement,
  withholding_tax_code,
  vertt,
  vertn,
  vbewa,
  mdflg,
  kfrst,
  status_bussiness,
  ecrtt,
  ecrtn,
  flow_type,
  currency_key
FROM ranked_data
WHERE row_num = 1
""")